In [1]:
import os 

os.chdir("../..")

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "mps" if torch.backends.mps.is_available() else "cpu"

# === 4. Load and preprocess data ===
def load_dataset(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['label'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'label']]
    return Dataset.from_pandas(df)

train_dataset = load_dataset(paths.arabic_data_dir / "train_ar.tsv")
val_dataset   = load_dataset(paths.arabic_data_dir / "dev_ar.tsv")
test_dataset  = load_dataset(paths.arabic_data_dir / "dev_test_ar.tsv")
competition_test_dataset = load_dataset(paths.arabic_data_dir / "test_ar_labeled.tsv")

W0615 18:59:50.296000 26588 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
# === 5. Tokenization ===
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)
competition_test_dataset = competition_test_dataset.map(tokenize_fn, batched=True)


train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")
competition_test_dataset = competition_test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
competition_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/2446 [00:00<?, ? examples/s]

Map:   0%|          | 0/467 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

Map:   0%|          | 0/1036 [00:00<?, ? examples/s]

In [3]:
# === 7. Define metrics ===
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

In [ ]:
# === 6. Load model and add LoRA ===

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True )

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query", "key", "value"]  
)

model = get_peft_model(model, lora_config).to(device)


# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results/arabic_lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# === 9. Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

# === 11. Evaluate on test set ===
trainer.evaluate(eval_dataset=test_dataset)

  0%|          | 0/6120 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.7509, 'grad_norm': 3.164447069168091, 'learning_rate': 4.991830065359477e-05, 'epoch': 0.02}
{'loss': 0.6906, 'grad_norm': 4.1256327629089355, 'learning_rate': 4.983660130718955e-05, 'epoch': 0.03}
{'loss': 0.7275, 'grad_norm': 5.46102237701416, 'learning_rate': 4.975490196078432e-05, 'epoch': 0.05}
{'loss': 0.7454, 'grad_norm': 5.023782253265381, 'learning_rate': 4.967320261437909e-05, 'epoch': 0.07}
{'loss': 0.6569, 'grad_norm': 8.242788314819336, 'learning_rate': 4.959150326797386e-05, 'epoch': 0.08}
{'loss': 0.7101, 'grad_norm': 4.897618770599365, 'learning_rate': 4.9509803921568634e-05, 'epoch': 0.1}
{'loss': 0.7061, 'grad_norm': 3.834735155105591, 'learning_rate': 4.9428104575163404e-05, 'epoch': 0.11}
{'loss': 0.7137, 'grad_norm': 7.145719051361084, 'learning_rate': 4.9346405228758174e-05, 'epoch': 0.13}
{'loss': 0.6853, 'grad_norm': 3.5212061405181885, 'learning_rate': 4.9264705882352944e-05, 'epoch': 0.15}
{'loss': 0.6901, 'grad_norm': 3.950662851333618, 'learning_r

  0%|          | 0/117 [00:00<?, ?it/s]

{'eval_loss': 0.6790615320205688, 'eval_accuracy': 0.5546038543897216, 'eval_f1_macro': 0.43761000463177396, 'eval_precision': 0.5020052866648437, 'eval_recall': 0.5008229529046496, 'eval_runtime': 21.3759, 'eval_samples_per_second': 21.847, 'eval_steps_per_second': 5.473, 'epoch': 1.0}


...

{'eval_loss': 0.7155539393424988, 'eval_accuracy': 0.5717344753747323, 'eval_f1_macro': 0.5532468526384265, 'eval_precision': 0.5572052315473368, 'eval_recall': 0.5541559121684809, 'eval_runtime': 20.8397, 'eval_samples_per_second': 22.409, 'eval_steps_per_second': 5.614, 'epoch': 10.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\peft\utils\other.py:611: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 50435b53-f6e4-49f5-aecd-cbd21036d18b)') - silently ignoring the lookup for the file config.json in CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment.
  warnings.warn(
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment - will assume that the vocabulary was not modified.
  warnings.warn(
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'train_runtime': 5313.7839, 'train_samples_per_second': 4.603, 'train_steps_per_second': 1.152, 'train_loss': 0.6537797230521059, 'epoch': 10.0}


  0%|          | 0/187 [00:00<?, ?it/s]

{'eval_loss': 0.7207608819007874,
 'eval_accuracy': 0.5655080213903744,
 'eval_f1_macro': 0.5495002992830179,
 'eval_precision': 0.5521187469645459,
 'eval_recall': 0.550030959752322,
 'eval_runtime': 31.8244,
 'eval_samples_per_second': 23.504,
 'eval_steps_per_second': 5.876,
 'epoch': 10.0}

In [8]:
from tqdm import tqdm
from peft import PeftModel

model_name = "CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment"
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True )

CHECKPOINT_PATH = "./results/arabic_lora/checkpoint-2448" 

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_PATH)

model = PeftModel.from_pretrained(base_model, CHECKPOINT_PATH)
model = model.to(device)
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (defaul

In [9]:
eval_args = TrainingArguments(
    output_dir="./results/temp_eval",
    per_device_eval_batch_size=16, 
    report_to="none",
)

eval_trainer = Trainer(
    model=model,
    args=eval_args,
    compute_metrics=compute_metrics,
)

print("\nRunning evaluation on the competition test set...")
results = eval_trainer.evaluate(eval_dataset=competition_test_dataset)


# === 5. Print the Final Results ===
print("\n--- Competition Test Set Results ---")
for key, value in results.items():
    # Example: 'eval_f1_macro' -> 'F1-Macro'
    formatted_key = key.replace("eval_", "").replace("_", "-").capitalize()
    print(f"{formatted_key}: {value:.4f}")
print("------------------------------------")


Running evaluation on the competition test set...


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/65 [00:00<?, ?it/s]


--- Competition Test Set Results ---
Loss: 0.6350
Accuracy: 0.6284
F1-macro: 0.5710
Precision: 0.5697
Recall: 0.5743
Runtime: 50.3276
Samples-per-second: 20.5850
Steps-per-second: 1.2920
------------------------------------
